<a href="https://colab.research.google.com/github/ruantos/python-para-data/blob/main/Learning_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the hard way

### Installing dependencies
1. install java
2. downloads spark tar
3. unzip it
4. pip install findspark


In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,168 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,103 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,518 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,270 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,207 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu ja

### Setting environment variables

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
            .builder \
            .appName("Primeira Sessão") \
            .getOrCreate()

## Mounting your google drive

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Unziping files

In [5]:
# import zipfile

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/estabelecimentos.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')

# zipfile \
#     .ZipFile('/content/drive/MyDrive/Colab Notebooks/Curso spark/socios.zip', 'r') \
#     .extractall('/content/drive/MyDrive/Colab Notebooks/Curso spark/')


## Creating sparks dataframes

In [6]:
path = {
    "empresas": "/content/drive/MyDrive/Colab Notebooks/Curso spark/empresas",
    "estabelecimentos": "/content/drive/MyDrive/Colab Notebooks/Curso spark/estabelecimentos.zip",
    "socios": "/content/drive/MyDrive/Colab Notebooks/Curso spark/socios"
    }

In [9]:
empresas = spark.read.csv(path["empresas"], sep=";", inferSchema=True)
# estabelecimentos = spark.read.csv(path["estabelecimentos"], sep=";", inferSchema=True)
socios = spark.read.csv(path["socios"], sep=";", inferSchema=True)

## Changing columns names

In [10]:
empresas.limit(5).show()

+----+--------------------+----+---+-------+---+----+
| _c0|                 _c1| _c2|_c3|    _c4|_c5| _c6|
+----+--------------------+----+---+-------+---+----+
| 306|FRANCAMAR REFRIGE...|2240| 49|   0,00|  1|NULL|
|1355|BRASILEIRO & OLIV...|2062| 49|   0,00|  5|NULL|
|4820|REGISTRO DE IMOVE...|3034| 32|   0,00|  5|NULL|
|5347|ROSELY APARECIDA ...|2135| 50|   0,00|  5|NULL|
|6846|BADU E FILHOS TEC...|2062| 49|4000,00|  1|NULL|
+----+--------------------+----+---+-------+---+----+



In [11]:
empresasColNames = ['cnpj_basico', 'razao_social_nome_empresarial', 'natureza_juridica', 'qualificacao_do_responsavel', 'capital_social_da_empresa', 'porte_da_empresa', 'ente_federativo_responsavel']
estabsColNames = ['cnpj_basico', 'cnpj_ordem', 'cnpj_dv', 'identificador_matriz_filial', 'nome_fantasia', 'situacao_cadastral', 'data_situacao_cadastral', 'motivo_situacao_cadastral', 'nome_da_cidade_no_exterior', 'pais', 'data_de_inicio_atividade', 'cnae_fiscal_principal', 'cnae_fiscal_secundaria', 'tipo_de_logradouro', 'logradouro', 'numero', 'complemento', 'bairro', 'cep', 'uf', 'municipio', 'ddd_1', 'telefone_1', 'ddd_2', 'telefone_2', 'ddd_do_fax', 'fax', 'correio_eletronico', 'situacao_especial', 'data_da_situacao_especial']
sociosColNames = ['cnpj_basico', 'identificador_de_socio', 'nome_do_socio_ou_razao_social', 'cnpj_ou_cpf_do_socio', 'qualificacao_do_socio', 'data_de_entrada_sociedade', 'pais', 'representante_legal', 'nome_do_representante', 'qualificacao_do_representante_legal', 'faixa_etaria']

In [12]:
def rename_col(df, col_names):
    for index, name in enumerate(col_names):
        # enumerate retorna uma tupla contendo
        # o indice e cada valor no array

        # params são o nome da coluna e o novo nome
        df = df.withColumnRenamed(f"_c{index}", name)
    return df

In [14]:
# empresas = rename_col(empresas, empresasColNames)
empresas = rename_col(empresas, empresasColNames)
# estabelecimentos = rename_col(estabelecimentos, estabsColNames)
socios = rename_col(socios, sociosColNames)

In [15]:
empresas.limit(5).show()

+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|cnpj_basico|razao_social_nome_empresarial|natureza_juridica|qualificacao_do_responsavel|capital_social_da_empresa|porte_da_empresa|ente_federativo_responsavel|
+-----------+-----------------------------+-----------------+---------------------------+-------------------------+----------------+---------------------------+
|        306|         FRANCAMAR REFRIGE...|             2240|                         49|                     0,00|               1|                       NULL|
|       1355|         BRASILEIRO & OLIV...|             2062|                         49|                     0,00|               5|                       NULL|
|       4820|         REGISTRO DE IMOVE...|             3034|                         32|                     0,00|               5|                       NULL|
|       5347|         ROSELY APARE

In [16]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social_nome_empresarial: string (nullable = true)
 |-- natureza_juridica: integer (nullable = true)
 |-- qualificacao_do_responsavel: integer (nullable = true)
 |-- capital_social_da_empresa: string (nullable = true)
 |-- porte_da_empresa: integer (nullable = true)
 |-- ente_federativo_responsavel: string (nullable = true)



## Using replace and changing cols datatypes

In [17]:
from pyspark.sql.types import DecimalType, StringType, DoubleType
from pyspark.sql import functions as f

In [18]:
col = "capital_social_da_empresa"
empresas = empresas.withColumn(col, f.regexp_replace(col, ",", "."))

In [19]:
empresas = empresas.withColumn(col, empresas[col].cast(DoubleType()))

## Converting string to date

In [20]:
socios = socios.withColumn("data_de_entrada_sociedade",\
           f.to_date(socios.data_de_entrada_sociedade.cast(StringType()), "yyyyMMdd"))
# df.withColumn(col, f.to_date(df[col].cast(old_type()), "date_formatt") )

In [33]:
socios.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- identificador_de_socio: integer (nullable = true)
 |-- nome_do_socio_ou_razao_social: string (nullable = true)
 |-- cnpj_ou_cpf_do_socio: string (nullable = true)
 |-- qualificacao_do_socio: integer (nullable = true)
 |-- data_de_entrada_sociedade: date (nullable = true)
 |-- pais: integer (nullable = true)
 |-- representante_legal: string (nullable = true)
 |-- nome_do_representante: string (nullable = true)
 |-- qualificacao_do_representante_legal: integer (nullable = true)
 |-- faixa_etaria: integer (nullable = true)



## SQL Queries

- Select
- Funções de data
- Alias
- Params do show

In [38]:
socios\
    .select("cnpj_basico" ,"identificador_de_socio",
            f.year("data_de_entrada_sociedade").alias("ano_entrada"),
            f.month("data_de_entrada_sociedade").alias("mes_entrada"))\
    .show(5, truncate=False)



+-----------+----------------------+-----------+-----------+
|cnpj_basico|identificador_de_socio|ano_entrada|mes_entrada|
+-----------+----------------------+-----------+-----------+
|        411|                     2|       1994|          7|
|        411|                     2|       1994|          7|
|       5813|                     2|       1994|          5|
|       5813|                     2|       1994|          5|
|      14798|                     2|       1994|          6|
+-----------+----------------------+-----------+-----------+
only showing top 5 rows



In [39]:
data = [
    ('GISELLE PAULA GUIMARAES CASTRO', 15),
    ('ELAINE GARCIA DE OLIVEIRA', 22),
    ('JOAO CARLOS ABNER DE LOURDES', 43),
    ('MARTA ZELI FERREIRA', 24),
    ('LAUDENETE WIGGERS ROEDER', 51)
]
colNames = ['nome', 'idade']
df = spark.createDataFrame(data, colNames)
df.show(truncate=False)

+------------------------------+-----+
|nome                          |idade|
+------------------------------+-----+
|GISELLE PAULA GUIMARAES CASTRO|15   |
|ELAINE GARCIA DE OLIVEIRA     |22   |
|JOAO CARLOS ABNER DE LOURDES  |43   |
|MARTA ZELI FERREIRA           |24   |
|LAUDENETE WIGGERS ROEDER      |51   |
+------------------------------+-----+



## Query com substring e concatenação

In [50]:
df.select(
    f.concat_ws(
        ", ",
        f.substring_index("nome", " ", 1),
        f.substring_index("nome", " ", -1)
        ).alias("ident"),
    "idade"
    )\
    .show(truncate=False)

+-----------------+-----+
|ident            |idade|
+-----------------+-----+
|GISELLE, CASTRO  |15   |
|ELAINE, OLIVEIRA |22   |
|JOAO, LOURDES    |43   |
|MARTA, FERREIRA  |24   |
|LAUDENETE, ROEDER|51   |
+-----------------+-----+



## Counting Nulls and NaN

In [57]:
socios\
    .select([f.count( f.when(f.isnull(c), 1)).alias(f"Qtd_nulls_em_{c}") for c in socios.columns])\
    .show()

+------------------------+-----------------------------------+------------------------------------------+---------------------------------+----------------------------------+--------------------------------------+-----------------+--------------------------------+----------------------------------+------------------------------------------------+-------------------------+
|Qtd_nulls_em_cnpj_basico|Qtd_nulls_em_identificador_de_socio|Qtd_nulls_em_nome_do_socio_ou_razao_social|Qtd_nulls_em_cnpj_ou_cpf_do_socio|Qtd_nulls_em_qualificacao_do_socio|Qtd_nulls_em_data_de_entrada_sociedade|Qtd_nulls_em_pais|Qtd_nulls_em_representante_legal|Qtd_nulls_em_nome_do_representante|Qtd_nulls_em_qualificacao_do_representante_legal|Qtd_nulls_em_faixa_etaria|
+------------------------+-----------------------------------+------------------------------------------+---------------------------------+----------------------------------+--------------------------------------+-----------------+-------------------

## Replacing NaN

In [63]:
# Se passar valor numérico, ele só preenche nas colunas numéricas
# O equivalente para outros types também se aplica
socios.na.fill("-").show()
socios.na.fill(0).show()

+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|cnpj_basico|identificador_de_socio|nome_do_socio_ou_razao_social|cnpj_ou_cpf_do_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|
+-----------+----------------------+-----------------------------+--------------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+
|        411|                     2|         LILIANA PATRICIA ...|         ***678188**|                   22|               1994-07-25|NULL|        ***000000**|                    -|                                  0|           7|
|        411|                     2|         CRISTINA HUNDERTMARK|      